# Transfer Learning CIFAR10

* Train a simple convnet on the CIFAR dataset the first 5 output classes [0..4].
* Freeze convolutional layers and fine-tune dense layers for the last 5 ouput classes [5..9].


In [0]:
from __future__ import absolute_import, division, print_function
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Reshape
from keras.layers import Convolution2D, MaxPooling2D, BatchNormalization
from keras.utils import np_utils
from keras import regularizers
import pickle
from matplotlib import pyplot as plt
import seaborn as sns

### 1. Import CIFAR10 data and create 2 datasets with one dataset having classes from 0 to 4 and other having classes from 5 to 9 

In [0]:
from keras.datasets import cifar10

In [0]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [0]:
X_train04 = X_train[np.where(y_train<5)[0]]
X_test04 = X_test[np.where(y_test<5)[0]]
y_train04 = y_train[y_train<5]
y_test04 = y_test[y_test<5]

In [0]:
X_train59 = X_train[np.where(y_train>4)[0]]
X_test59 = X_test[np.where(y_test>4)[0]]
y_train59 = y_train[y_train>4]
y_test59 = y_test[y_test>4]

In [41]:
y_train04.shape

(25000,)

In [42]:
X_train04.shape

(25000, 32, 32, 3)

In [43]:
y_train59.shape

(25000,)

In [44]:
X_train59.shape

(25000, 32, 32, 3)

### 2. Use One-hot encoding to divide y_train and y_test into required no of output classes

In [0]:
X_train04 = X_train04.reshape(X_train04.shape[0], 32, 32, 3).astype('float32')
X_test04 = X_test04.reshape(X_test04.shape[0], 32, 32, 3).astype('float32')
X_train04 /= 255
X_test04 /= 255
y_train04 = np_utils.to_categorical(y_train04, 5)
y_test04 = np_utils.to_categorical(y_test04, 5)

In [0]:
y_train59 = y_train59-5
y_test59 = y_test59-5

In [0]:
X_train59 = X_train59.reshape(X_train59.shape[0], 32, 32, 3).astype('float32')
X_test59 = X_test59.reshape(X_test59.shape[0], 32, 32, 3).astype('float32')
X_train59 /= 255
X_test59 /= 255
y_train59 = np_utils.to_categorical(y_train59, 5)
y_test59 = np_utils.to_categorical(y_test59, 5)

### 3. Build a sequential neural network model which can classify the classes 0 to 4 of CIFAR10 dataset with at least 80% accuracy on test data

In [48]:
model = Sequential()

#1st Con layer
model.add(Convolution2D(32, 3, 3, input_shape=(32, 32, 3)))
model.add(Activation('relu'))

#2st Con layer
model.add(Convolution2D(32, 3, 3))
model.add(Activation('relu'))

# Max Pooling
model.add(MaxPooling2D(pool_size=(2,2)))
    
# Dropout
model.add(Dropout(0.2))
model.add(BatchNormalization())

#3rd Con layer
model.add(Convolution2D(32, 3, 3))
model.add(Activation('relu'))

# Dropout
model.add(Dropout(0.2))
model.add(BatchNormalization())
#model.add(Dense(64, input_dim=64, kernel_regularizer = regularizers.l2(0.01)))

# Fully Connected Layer
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))

# Dropout
model.add(Dropout(0.2))
model.add(BatchNormalization())

#Prediction Layer
model.add(Dense(5))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

#Early stopping
early_stopping = keras.callbacks.EarlyStopping(monitor='val_acc', patience=5, verbose=1, mode='auto')
callback_list = [early_stopping]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(32, 32, 3...)`
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3))`
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3))`


In [49]:
model.fit(X_train04, y_train04, batch_size= 128, nb_epoch= 10, validation_data=(X_test04, y_test04), callbacks=callback_list)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


Train on 25000 samples, validate on 5000 samples
Epoch 1/10
25000/25000 [==============================] - 7s 298us/step - loss: 1.0376 - acc: 0.5972 - val_loss: 0.9999 - val_acc: 0.6214
Epoch 2/10
25000/25000 [==============================] - 6s 251us/step - loss: 0.7685 - acc: 0.7030 - val_loss: 0.7231 - val_acc: 0.7212
Epoch 3/10
25000/25000 [==============================] - 6s 245us/step - loss: 0.6434 - acc: 0.7556 - val_loss: 0.6495 - val_acc: 0.7454
Epoch 4/10
25000/25000 [==============================] - 6s 242us/step - loss: 0.5641 - acc: 0.7867 - val_loss: 0.6400 - val_acc: 0.7708
Epoch 5/10
25000/25000 [==============================] - 6s 236us/step - loss: 0.5061 - acc: 0.8088 - val_loss: 0.6576 - val_acc: 0.7548
Epoch 6/10
25000/25000 [==============================] - 6s 240us/step - loss: 0.4504 - acc: 0.8310 - val_loss: 0.5388 - val_acc: 0.8062
Epoch 7/10
25000/25000 [==============================] - 6s 237us/step - loss: 0.4066 - acc: 0.8484 - val_loss: 0.5969 - v

In [50]:
loss_and_metrics = model.evaluate(X_test04, y_test04)
print(loss_and_metrics)

5000/5000 [==============================] - 1s 135us/step
[0.5370065430164337, 0.811]


### 4. In the model which was built above (for classification of classes 0-4 in CIFAR10), make only the dense layers to be trainable and conv layers to be non-trainable

In [51]:
#Freezing layers in the model which don't have 'dense' in their name
for layer in model.layers:
  if('dense' not in layer.name): #prefix detection to freeze layers which does not have dense
    #Freezing a layer
    layer.trainable = False

#Module to print colourful statements
from termcolor import colored

#Check which layers have been frozen 
for layer in model.layers:
  print (colored(layer.name, 'blue'))
  print (colored(layer.trainable, 'red'))

conv2d_4
False
activation_6
False
conv2d_5
False
activation_7
False
max_pooling2d_2
False
dropout_4
False
batch_normalization_4
False
conv2d_6
False
activation_8
False
dropout_5
False
batch_normalization_5
False
flatten_2
False
dense_3
True
activation_9
False
dropout_6
False
batch_normalization_6
False
dense_4
True
activation_10
False


### 5. Utilize the the model trained on CIFAR 10 (classes 0 to 4) to classify the classes 5 to 9 of CIFAR 10  (Use Transfer Learning) <br>
Achieve an accuracy of more than 85% on test data

In [52]:
model.fit(X_train59, y_train59, batch_size= 128, nb_epoch= 10, validation_data=(X_test59, y_test59))

Train on 25000 samples, validate on 5000 samples
Epoch 1/10
  896/25000 [>.............................] - ETA: 5s - loss: 4.6046 - acc: 0.1808

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


25000/25000 [==============================] - 6s 240us/step - loss: 0.8516 - acc: 0.7276 - val_loss: 0.5106 - val_acc: 0.8170
Epoch 2/10
25000/25000 [==============================] - 6s 237us/step - loss: 0.4215 - acc: 0.8470 - val_loss: 0.3980 - val_acc: 0.8576
Epoch 3/10
25000/25000 [==============================] - 6s 236us/step - loss: 0.3432 - acc: 0.8740 - val_loss: 0.4130 - val_acc: 0.8508
Epoch 4/10
25000/25000 [==============================] - 6s 235us/step - loss: 0.3003 - acc: 0.8900 - val_loss: 0.3789 - val_acc: 0.8650
Epoch 5/10
25000/25000 [==============================] - 6s 237us/step - loss: 0.2552 - acc: 0.9078 - val_loss: 0.3291 - val_acc: 0.8874
Epoch 6/10
25000/25000 [==============================] - 6s 239us/step - loss: 0.2219 - acc: 0.9207 - val_loss: 0.3301 - val_acc: 0.8766
Epoch 7/10
25000/25000 [==============================] - 6s 237us/step - loss: 0.1973 - acc: 0.9290 - val_loss: 0.3564 - val_acc: 0.8746
Epoch 8/10
25000/25000 [=====================

In [53]:
loss_and_metrics = model.evaluate(X_test59, y_test59)
print(loss_and_metrics)

5000/5000 [==============================] - 1s 135us/step
[0.33218528892993926, 0.8856]


# Text classification using TF-IDF

### 6. Load the dataset from sklearn.datasets

In [0]:
from sklearn.datasets import fetch_20newsgroups

In [0]:
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']

### 7. Training data

In [0]:
twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)

### 8. Test data

In [0]:
twenty_test = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=42)

###  a.  You can access the values for the target variable using .target attribute 
###  b. You can access the name of the class in the target variable with .target_names


In [58]:
twenty_train.target

array([1, 1, 3, ..., 2, 2, 2])

In [59]:
twenty_train.target_names

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

In [60]:
twenty_train.data[0:5]

['From: sd345@city.ac.uk (Michael Collier)\nSubject: Converting images to HP LaserJet III?\nNntp-Posting-Host: hampton\nOrganization: The City University\nLines: 14\n\nDoes anyone know of a good way (standard PC application/PD utility) to\nconvert tif/img/tga files into LaserJet III format.  We would also like to\ndo the same, converting to HPGL (HP plotter) files.\n\nPlease email any response.\n\nIs this the correct group?\n\nThanks in advance.  Michael.\n-- \nMichael Collier (Programmer)                 The Computer Unit,\nEmail: M.P.Collier@uk.ac.city                The City University,\nTel: 071 477-8000 x3769                      London,\nFax: 071 477-8565                            EC1V 0HB.\n',
 "From: ani@ms.uky.edu (Aniruddha B. Deglurkar)\nSubject: help: Splitting a trimming region along a mesh \nOrganization: University Of Kentucky, Dept. of Math Sciences\nLines: 28\n\n\n\n\tHi,\n\n\tI have a problem, I hope some of the 'gurus' can help me solve.\n\n\tBackground of the probl

### 9.  Now with dependent and independent data available for both train and test datasets, using TfidfVectorizer fit and transform the training data and test data and get the tfidf features for both

In [0]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import pandas as pd

In [0]:
ve = TfidfVectorizer()
train = ve.fit(twenty_train.data)
vedata = ve.transform(twenty_train.data)
df = pd.DataFrame(vedata.todense(),columns=ve.get_feature_names())

In [63]:
df.head()

,00,000,0000,0000001200,000005102000,0001,000100255pixel,00014,000406,0007,000usd,0010,001004,0010580b,001125,001200201pixel,0014,001642,00196,002,0028,003258u19250,0033,0038,0039,004021809,004158,004627,0049,00500,005148,00630,008561,0094,00am,00index,00pm,01,0100,010116,...,zoerasterism,zola,zolf,zolft,zoloft,zombie,zonal,zone,zonker,zoo,zooid,zool,zoom,zooming,zooms,zopfi,zorasterism,zorg,zorn,zrz,zsoft,zt,zubkoff,zues,zug,zumder,zur,zurich,zurlo,zus,zvi,zvonko,zwart,zyeh,zyklon,zyxel,zz,zzz,ªl,íålittin
0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.29521,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.055283,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.057185,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
test = ve.transform(twenty_test.data)
df1 = pd.DataFrame(test.todense(), columns=ve.get_feature_names())

In [65]:
df1.head()

,00,000,0000,0000001200,000005102000,0001,000100255pixel,00014,000406,0007,000usd,0010,001004,0010580b,001125,001200201pixel,0014,001642,00196,002,0028,003258u19250,0033,0038,0039,004021809,004158,004627,0049,00500,005148,00630,008561,0094,00am,00index,00pm,01,0100,010116,...,zoerasterism,zola,zolf,zolft,zoloft,zombie,zonal,zone,zonker,zoo,zooid,zool,zoom,zooming,zooms,zopfi,zorasterism,zorg,zorn,zrz,zsoft,zt,zubkoff,zues,zug,zumder,zur,zurich,zurlo,zus,zvi,zvonko,zwart,zyeh,zyklon,zyxel,zz,zzz,ªl,íålittin
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 10. Use logisticRegression with tfidf features as input and targets as output and train the model and report the train and test accuracy score

In [66]:
y = twenty_train.target
y_test = twenty_test.target
LR = LogisticRegression()
model1 = LR.fit(df,y)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [67]:
y_pred = LR.predict(df)
y_pred

array([1, 1, 3, ..., 2, 2, 2])

In [68]:
y_pred1 = LR.predict(df1)
y_pred1

array([2, 2, 2, ..., 2, 2, 1])

In [69]:
print('Training Accuracy', metrics.accuracy_score(y, y_pred))

Training Accuracy 0.9827204253433761


In [70]:
print('Test Accuracy', metrics.accuracy_score(y_test, y_pred1))

Test Accuracy 0.8868175765645806
